In [1]:
#! pip install -U pip setuptools wheel
#! pip install -U 'spacy[cuda12x]'
#! python -m spacy download en_core_web_trf
#! python -m spacy download en_core_web_sm
#! pip install pandas

### Baixar CSV DAtaset

Para realizar o tratamento de dados a seguir, o IMDB Dataset deve ser baixado, e colocado na pasta raiz do projeto

<a href="https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews" target="_blank">Conjunto de dados IMDB de 50 mil críticas de filmes</a>


In [2]:
import re
import spacy
from spacy.tokens import Doc
import pandas as pd
from tqdm import tqdm


In [3]:
# Carregar o modelo em inglês
# nlp = spacy.load("en_core_web_trf", exclude=["parser", "ner"])
nlp = spacy.load("en_core_web_sm", exclude=["parser", "ner"])
file_path = "D:/tcc2/guilherme/IMDB Dataset.csv"
df = pd.read_csv(file_path)
# df = df[:1000]

In [4]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [5]:
df["sentiment"].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [6]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [7]:
df.duplicated().sum()

418

In [8]:
# Remove linhas de reviews_treated duplicadoss
df = df.drop_duplicates()

In [9]:
df.duplicated().sum()

0

In [10]:
df["sentiment"].value_counts()

sentiment
positive    24884
negative    24698
Name: count, dtype: int64

In [11]:
def obter_maior_texto(series):
    return max(text.split() for text in series)

In [12]:
maior_string = max(df["review"], key=lambda x: len(x.split()))
tam_maior_string = len(maior_string)
print(tam_maior_string)

13704


In [13]:
# funcoes de tratamento comuns
def remove_extra_spaces(text):
    r"Remover espacos do inicio e fim de frases e ubstituir sequências de espaços por um espaço"
    return re.sub(" +", " ", text).strip()


def remove_tags(raw_text):
    cleaned_text = re.sub(re.compile("<.*?>"), "", raw_text)
    return cleaned_text


def normalize_lowercase(text: str) -> str:
    return text.lower()


# funcoes de tratamento spacy
def lemmatize_text_remove_stopwords_and_puntuaction(doc: Doc | str):
    if not isinstance(doc, Doc):
        doc = nlp(doc)
    return " ".join(
        [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
    )


def remove_stopwords_and_puntuaction(doc: Doc | str):
    if not isinstance(doc, Doc):
        doc = nlp(doc)
    return " ".join(
        [token.text for token in doc if not token.is_stop and not token.is_punct]
    )


# def remove_punctuation(doc):
#     return " ".join([token.text for token in doc if not token.is_punct])


# def remove_stopwords(doc):
#     return " ".join([token.text for token in doc if not token.is_stop])


# def lemmatize_text(doc):
#     return " ".join([token.lemma_ for token in doc])

In [14]:
# Aplicar as funções de str ao DataFrame 
reviews_treated = df["review"]

reviews_treated = reviews_treated.apply(remove_tags)
reviews_treated = reviews_treated.apply(normalize_lowercase)
reviews_treated = reviews_treated.apply(remove_extra_spaces)


In [15]:
len_texts = len(reviews_treated)
# Processar os textos com nlp.pipe e atualizar a barra de progresso a cada iteração
docs = list(
    tqdm(
        nlp.pipe(
            reviews_treated,
            batch_size=500,
            disable=["parser", "ner"],
        ),
        colour="green",
        desc="Progresso: ",
        total=len_texts,
    )
)

Progresso: 100%|██████████| 49582/49582 [09:17<00:00, 88.89it/s] 


In [18]:
reviews_treated = [lemmatize_text_remove_stopwords_and_puntuaction(doc) for doc in docs]

In [40]:
# reviews_treated = [remove_stopwords_and_puntuaction(doc) for doc in docs]
# durou 3.3s  em 50k arquivos

In [19]:
reviews_treated = [remove_extra_spaces(text) for text in reviews_treated]

100%|██████████| 49582/49582 [00:01<00:00, 41609.39it/s]


In [20]:
df["review"]= reviews_treated

In [21]:
maior_string = max(df["review"], key=lambda x: len(x.split()))
tam_maior_string= len(maior_string)
print(tam_maior_string)

8532


In [22]:
df.to_csv("IMDB_Dataset_tratado_lemma.csv", index=False)